In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [55]:
def scrape_coches_ocasion(url, diccionario):

    llamada = requests.get(url)  # Realiza una solicitud GET a la URL proporcionada y almacena la respuesta en 'llamada'.

    if llamada.status_code != 200:  # Comprueba si la respuesta no fue exitosa (código de estado distinto de 200).
            print(f"El motivo REASON por el que la llamada falló es {llamada.reason}")  # Imprime la razón de la falla.
            print(f"El motivo TEXT por el que la llamada falló es {llamada.text}")  # Imprime la razón de la falla.
            return None
    else:
        soup = BeautifulSoup(llamada.text, 'html.parser')
        spans = soup.find_all('span', {"class": "cardVehicle_spot__e6YZx"}) # Sopa de coches y precios originales
        precios_descuento = soup.find_all('span', {"class":"cardVehicle_finance__SG6JV"}) # Sopa de precios con descuento
        km = soup.find_all('span', {"class":"characteristics_elements__Mb1S_"}) # Sopa de km

        for kilom in km: # Metiendo los Km en el diccionario
            if kilom.get_text('data-test').endswith('Km'):
                diccionario['km'].append(kilom.get_text('data-test'))

        for desc in precios_descuento: # Metiendo los precios con descuento en el diccionario
            if "€" in desc.text:
                diccionario['precio_rebajado'].append(desc.text)

        for nombre in spans: # Metiendo los coches en el diccionario
            if nombre.get('data-test') == 'span-brand-model':
                diccionario['coche'].append(nombre.text)

        for precio in spans: # Metiendo los precios originales en el diccionario
            try:
                diccionario['precio_original'].append(precio.text)
            except:
                diccionario['precio_original'].append('Null')


        # Verificar que todas las listas tengan el mismo len
        if len(diccionario['coche']) == len(diccionario['precio_original']) == len(diccionario['precio_rebajado']) == len(diccionario['km']):
            print('OK')
        else: # Si no tienen el mismo len, buscar el problema y corregirlo
            print('Lens diferentes')
        #     coches_precios_copy = [i.text for i in coches_precios]    
        #     counter = 0
        #     for i in coches_precios_copy:
        #         counter += 1
        #         if counter %2 == 0 and "€" not in i:
        #             index = counter -1
        #             coches_precios_copy.insert(index, "value")
        #         else:
        #             pass
        #     counter = 0
        #     for valor in coches_precios_copy:
        #         counter+=1
        #         if valor == 'value':
        #             index = counter//2-1
        #             diccionario['precio_original'].insert(index, diccionario['precio_rebajado'][index])
                  
        return diccionario#, pd.DataFrame(diccionario)  


url = "https://www.ocasionplus.com/coches-ocasion?v2&orderBy=morePopular"
coches = {'coche':[], 'precio_rebajado':[], 'precio_original':[], 'km':[]} # Crea diccionario para almacenar todos los datos del DataFrame
diccio = scrape_coches_ocasion(url, coches)


Lens diferentes


In [45]:
llamada = requests.get(url)
soup = BeautifulSoup(llamada.text, 'html.parser')
coches = soup.find_all('span', {"class":"cardVehicle_spot__e6YZx"}) # Sopa de coches
precios_originales = soup.find_all('span', {"class":"cardVehicle_spot__e6YZx"}) # Sopa de precios originales

In [43]:
coches[0].get_text('span', {"data-test":"span-brand-model"})
   

'Ford Kuga'

In [47]:
precios_originales[1].get_text('span', {"data-test":"span-price"})

'20.300€'

In [48]:
spans = soup.find_all('span', {"class": "cardVehicle_spot__e6YZx"})
coches = [span.text for span in spans if span.get('data-test') == 'span-brand-model']
precios_originales = [span.text for span in spans if span.get('data-test') == 'span-price']


In [49]:
coches

['Ford Kuga',
 'Nissan QASHQAI',
 'Mercedes GLA',
 'Opel Corsa',
 'Mini Countryman',
 'Lexus RX',
 'KIA Ceed',
 'Mazda CX-5',
 'Seat Ibiza',
 'Nissan X-TRAIL',
 'BMW Serie 2 Active Tourer',
 'Mercedes Clase A',
 'Renault Arkana',
 'Nissan Micra',
 'Audi TT',
 'Cupra Ateca',
 'Volkswagen Caravelle',
 'Mercedes Clase GLK',
 'Land Rover Range Rover Evoque',
 'Renault Mégane']

In [50]:
precios_originales

['20.300€',
 '15.990€',
 '10.460€',
 '16.150€',
 '32.490€',
 '25.590€',
 '18.300€',
 '29.490€',
 '25.990€',
 '9.350€',
 '33.900€',
 '34.900€',
 '13.500€']

In [ ]:
# <span class="cardVehicle_spot__e6YZx" data-test="span-brand-model">Nissan QASHQAI</span>
# <span class="cardVehicle_spot__e6YZx" data-test="span-price">15.990€</span>

In [58]:
len(diccio['coche']) == len(diccio['precio_rebajado']) == len(diccio['precio_original']) == len(diccio['km'])

False

In [59]:
print(len(diccio['coche']))
print(len(diccio['precio_rebajado']))
print(len(diccio['precio_original']))
print(len(diccio['km']))

20
20
35
20


In [14]:
llamada = requests.get(url)
soup = BeautifulSoup(llamada.text, 'html.parser')
coches_precios = soup.find_all('span', {"class":"cardVehicle_spot__e6YZx"})

In [29]:
for coche in coches_precios:
    print(coche.text)


Nissan QASHQAI
15.990€
Opel Corsa
10.460€
Lexus RX
32.490€
KIA Ceed
25.590€
Seat Ibiza
Nissan X-TRAIL
18.300€
BMW Serie 2 Active Tourer
Mercedes Clase A
29.490€
Renault Arkana
25.990€
Nissan Micra
9.350€
Audi TT
Cupra Ateca
33.900€
Volkswagen Caravelle
34.900€
Mercedes Clase GLK
Land Rover Range Rover Evoque
Renault Mégane
13.500€
Seat Ibiza
11.350€
Mercedes Clase GLK
Seat Ateca
25.900€
Citroën C3
12.700€


In [30]:
# for nombre_coche in coches_precios:
#     if '€' not in nombre_coche.text:
#         nombre = nombre_coche

for precio in coches_precios[1::2]:
    if '€' not in precio.text:
        precio = None
    print(precio.text)




15.990€
10.460€
32.490€
25.590€
Nissan X-TRAIL
BMW Serie 2 Active Tourer
29.490€
25.990€
9.350€
Cupra Ateca
Volkswagen Caravelle
Mercedes Clase GLK
Renault Mégane
Seat Ibiza
Mercedes Clase GLK
25.900€
12.700€


In [18]:
llamada = requests.get(url)
soup = BeautifulSoup(llamada.text, 'html.parser')
precios_descuento = soup.find_all('span', {"class":"cardVehicle_finance__SG6JV"})

In [19]:
for i in precios_descuento:
    print(i.text)

dCi 110 S&S Acenta 4x2 (110 CV)
14.536€
1.2 XEL Edition (75 CV)
9.509€
450h L Luxury Auto 4X4 (313 CV)
29.536€
1.6 T-GDi GT (204 CV)
23.263€
1.2 TSI Reference ITech (85 CV)
9.390€
1.6 DIG-T N-Connecta 4x2 7 plazas (163 CV)
16.636€
218d Active Tourer (150 CV)
11.490€
Mercedes-Benz A 200 (163 CV) Pack AMG
26.809€
Intens TCe (140 CV) EDC Microhíbrido
23.627€
1.2G Acenta (80 CV)
8.500€
Coupe 1.8 TFSI (160 CV)
13.950€
2.0 TSI S&S 4Drive DSG (300 CV)
30.818€
Trendline Batalla Corta 2.0 TDI BMT(102 CV) CAMPERIZADO
31.727€
Mercedes-Benz 350 Blue Efficiency 4Matic (306 CV)
14.500€
2.2L SD4 Prestige 4x4 Auto (190 CV)
17.500€
Intens Energy TCe (130 CV)
12.272€
1.0 EcoTSI S&S Style (95 CV)
10.318€
Mercedes-Benz 220 CDI 4Matic (170 CV)
20.500€
1.5 TSI X-Perience XM (150 CV)
23.545€
PureTech 110 S&S Shine (110 CV)
11.545€
